In [61]:
import pandas as pd
from sqlalchemy import create_engine


### Store CSV into DataFrame

In [62]:
from secrets import username, password


Read in and put into a database the Kroger products

In [63]:
csv_file = "../kroger/kroger_milk_prices.csv"
kroger_data_df = pd.read_csv(csv_file)
kroger_data_df.head()


,Unnamed: 0,productId,categories,brand,description,size,price.regular,price.promo,Date,Store_Id,image
0,0,1111040101,['Dairy'],Kroger,Kroger® Vitamin D Whole Milk,1 gal,2.99,0.0,02/08/2021,3400312,https://www.kroger.com/product/images/xlarge/f...
1,1,1111041600,['Dairy'],Kroger,Kroger® 2% Reduced Fat Milk,1/2 gal,1.99,0.0,02/08/2021,3400312,https://www.kroger.com/product/images/xlarge/f...
2,2,1111041660,['Dairy'],Kroger,Kroger® 1% Lowfat Milk,1 gal,2.99,0.0,02/08/2021,3400312,https://www.kroger.com/product/images/xlarge/f...
3,3,1111040601,['Dairy'],Kroger,Kroger® Vitamin D Whole Milk,1/2 gal,1.99,0.0,02/08/2021,3400312,https://www.kroger.com/product/images/xlarge/f...
4,4,1111042315,['Dairy'],Kroger,Kroger® Fat Free Skim Milk,1 gal,2.99,0.0,02/08/2021,3400312,https://www.kroger.com/product/images/xlarge/f...


Read in and put into a database HEB Products

In [64]:
path = "../data/heb_clean.csv"
heb_data_df = pd.read_csv(path)
heb_milk = heb_data_df.drop(columns=['coupon', 'name' , 'price', 'uomSalePrice', 'simple_type'])
milk_type_file = "../data/heb_milk_vs_types.csv"
milk_type_df = pd.read_csv(milk_type_file)
heb_milk = heb_milk.merge(milk_type_df, on = 'id')
heb_milk = heb_milk.rename(columns= {'id':'product_id' , 'brand' : 'Brand'})
heb_milk['Store_Number'] = 1111111
today = pd.datetime.now().date()
heb_data_df['Date'] = today
#heb_data_df.head()
heb_milk = heb_milk.drop(columns = ['type_x','types'])
heb_milk = heb_milk.rename(columns = {'type_y' : 'type'})
heb_milk


<ipython-input-64-b9708622fa7d>:9: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  today = pd.datetime.now().date()


,Brand,category,features,product_id,image,size,name,type,Store_Number
0,Central Market,Dairy & Eggs/Milk,"[' ORGANIC', ' GO LOCAL']",931316,https://images.heb.com/is/image/HEBGrocery/prd...,1 gal,Organics Whole Milk,whole,1111111
1,Central Market,Dairy & Eggs/Milk,"[' ORGANIC', ' GO LOCAL']",931317,https://images.heb.com/is/image/HEBGrocery/prd...,1 gal,Organics Reduced Fat 2% Milkfat Milk,fat,1111111
2,H-E-B,Dairy & Eggs/Milk,"[' GO LOCAL', ' SELECT INGREDIENTS', ' Primo...",671076,https://images.heb.com/is/image/HEBGrocery/prd...,1/2 gal,Select Ingredients MooTopia Lactose Free 2% R...,lactose,1111111
3,H-E-B,Dairy & Eggs/Milk,"[' CARB AWARE', ' LOW SATURATED FAT', ' GO ...",314136,https://images.heb.com/is/image/HEBGrocery/prd...,1/2 gal,Select Ingredients Fat Free Milk,fat,1111111
4,H-E-B,Dairy & Eggs/Milk,"[' GO LOCAL', ' SELECT INGREDIENTS']",314134,https://images.heb.com/is/image/HEBGrocery/prd...,1/2 gal,Select Ingredients 1% Low Fat Milk,fat,1111111
...,...,...,...,...,...,...,...,...,...
255,Hill Country Fare,Dairy & Eggs/Milk,[' GO LOCAL'],314131,https://images.heb.com/is/image/HEBGrocery/prd...,1 gal,Reduced Fat 2% Milk,fat,1111111
256,Hill Country Fare,Dairy & Eggs/Milk,[' GO LOCAL'],314124,https://images.heb.com/is/image/HEBGrocery/prd...,1 gal,Whole Milk,whole,1111111
257,Hill Country Fare,Dairy & Eggs/Milk,"[' CARB AWARE', ' LOW SATURATED FAT', ' GO ...",314128,https://images.heb.com/is/image/HEBGrocery/prd...,1 gal,Fat Free Milk,fat,1111111
258,H-E-B,Dairy & Eggs/Milk,"[' GO LOCAL', ' SELECT INGREDIENTS']",658779,https://images.heb.com/is/image/HEBGrocery/prd...,1 gal,Chocolate Milk,chocolate,1111111


Create dataframe to match Milk database table. Merge HEB and Kroger milk dataframes.

In [65]:
milk_df = kroger_data_df
milk_df = milk_df.drop(columns=['Unnamed: 0', 'description' , 'price.regular', 'price.promo', 'Date'])
features = 'none'
milk_df['features'] = features
kroger_milk_type_file = "../data/kroger_milk_vs_types.csv"
kroger_milk_type_df = pd.read_csv(kroger_milk_type_file)
kroger_milk_type_df = kroger_milk_type_df.rename(columns = {'id' : 'productId'})
milk_df = milk_df.merge(kroger_milk_type_df, on = 'productId')
milk_df = milk_df.drop(columns=['types'])
milk_df = milk_df.rename(columns={"productId": "product_id", "brand": "Brand" , "categories" : "category" , 'Store_Id' : 'Store_Number'})
complete_milk = pd.concat([milk_df, heb_milk])
complete_milk


,product_id,category,Brand,size,Store_Number,image,features,name,type
0,1111040101,['Dairy'],Kroger,1 gal,3400312,https://www.kroger.com/product/images/xlarge/f...,none,Kroger® Vitamin D Whole Milk,whole
1,1111041600,['Dairy'],Kroger,1/2 gal,3400312,https://www.kroger.com/product/images/xlarge/f...,none,Kroger® 2% Reduced Fat Milk,fat
2,1111041660,['Dairy'],Kroger,1 gal,3400312,https://www.kroger.com/product/images/xlarge/f...,none,Kroger® 1% Lowfat Milk,fat
3,1111040601,['Dairy'],Kroger,1/2 gal,3400312,https://www.kroger.com/product/images/xlarge/f...,none,Kroger® Vitamin D Whole Milk,whole
4,1111042315,['Dairy'],Kroger,1 gal,3400312,https://www.kroger.com/product/images/xlarge/f...,none,Kroger® Fat Free Skim Milk,fat
...,...,...,...,...,...,...,...,...,...
255,314131,Dairy & Eggs/Milk,Hill Country Fare,1 gal,1111111,https://images.heb.com/is/image/HEBGrocery/prd...,[' GO LOCAL'],Reduced Fat 2% Milk,fat
256,314124,Dairy & Eggs/Milk,Hill Country Fare,1 gal,1111111,https://images.heb.com/is/image/HEBGrocery/prd...,[' GO LOCAL'],Whole Milk,whole
257,314128,Dairy & Eggs/Milk,Hill Country Fare,1 gal,1111111,https://images.heb.com/is/image/HEBGrocery/prd...,"[' CARB AWARE', ' LOW SATURATED FAT', ' GO ...",Fat Free Milk,fat
258,658779,Dairy & Eggs/Milk,H-E-B,1 gal,1111111,https://images.heb.com/is/image/HEBGrocery/prd...,"[' GO LOCAL', ' SELECT INGREDIENTS']",Chocolate Milk,chocolate


Create and populate Store dataframe for Store database table. Will be automated once more stores are added in future itterations of project.

In [66]:
store_df = pd.DataFrame()
store_id = 3400312 , 1111111
store_df["Store_Number"] = store_id
zipcode = 77007 , 77007
store_df['Store_Zipcode'] = zipcode
name = "Kroger" , "HEB"
store_df['Store_Name'] = name
store_df.head()

,Store_Number,Store_Zipcode,Store_Name
0,3400312,77007,Kroger
1,1111111,77007,HEB


Read in and create dataframe for milk types. Formatted for database table.



In [67]:
milk_type_file = "../data/milk_types.csv"
milk_type_df = pd.read_csv(milk_type_file)
milk_type_df['Full_Name'] = 'Almond-Milk' , 'Chocolate-Milk', 'Reduced-Fat-Milk', 'Lactose-Free-Milk', 'Oat-Milk', 'Organic-Milk', 'Other', 'Powder-Milk', 'Protein-Product' , 'Whole'
milk_type_df = milk_type_df.rename(columns={'count_id' : 'count'})
milk_type_df

,type,count,Full_Name
0,almond,44,Almond-Milk
1,chocolate,43,Chocolate-Milk
2,fat,51,Reduced-Fat-Milk
3,lactose,19,Lactose-Free-Milk
4,oat,13,Oat-Milk
5,organic,10,Organic-Milk
6,other,47,Other
7,powder,3,Powder-Milk
8,protein,9,Protein-Product
9,whole,21,Whole


### Connect to local database

In [68]:
rds_connection_string = f"{username}:{password}@localhost:5432/milk_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [69]:
engine.table_names()

['Store', 'Milk', 'Milk_Type', 'Price_History']

Use drop duplicates to only add updated data to a dataframe that is then loaded into the database

In [70]:
temp_store = pd.read_sql_query('select * from "Store"', con=engine)
temp_store = temp_store.drop(columns = 'id')
#temp_store

In [71]:
db_store = pd.concat([temp_store, store_df]).drop_duplicates(keep=False)
db_store = db_store.reset_index(drop=True)   
#db_store.head()

### Use pandas to load csv converted DataFrame into database

In [72]:
db_store.to_sql(name='Store', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the customer_name table
* NOTE: can also check using pgAdmin

In [73]:
pd.read_sql_query('select * from "Store"', con=engine).head()

,id,Store_Number,Store_Name,Store_Zipcode
0,1,3400312,Kroger,77007
1,2,1111111,HEB,77007


Same process as above for store table is followed for milk type.

In [74]:
temp_type = pd.read_sql_query('select * from "Milk_Type"', con=engine)
temp_type = temp_type.drop(columns = 'id')
db_type = pd.concat([temp_type, milk_type_df]).drop_duplicates(keep=False)
db_type = db_type.reset_index(drop=True)   
db_type

,type,Full_Name,count


In [75]:
db_type.to_sql(name='Milk_Type', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the customer_location table

In [76]:
pd.read_sql_query('select * from "Milk_Type"', con=engine)

,id,type,Full_Name,count
0,1,almond,Almond-Milk,44
1,2,chocolate,Chocolate-Milk,43
2,3,fat,Reduced-Fat-Milk,51
3,4,lactose,Lactose-Free-Milk,19
4,5,oat,Oat-Milk,13
5,6,organic,Organic-Milk,10
6,7,other,Other,47
7,8,powder,Powder-Milk,3
8,9,protein,Protein-Product,9
9,10,whole,Whole,21


Retrieves and merges Store data with milk data to set the Store ID then next cell deletes data not required in Milk Table

In [77]:
stores2_df = pd.read_sql_query('select * from "Store"', con=engine).head()
complete_milk = complete_milk.merge(stores2_df, on = 'Store_Number')
#complete_milk.head()

In [78]:
complete_milk = complete_milk.drop(columns = ['Store_Name' , 'Store_Zipcode', 'Store_Number'])
#complete_milk.head()

In [79]:
complete_milk = complete_milk.rename(columns ={'id' : 'Store_ID'})
#complete_milk.head()

In [80]:
type2_df =pd.read_sql_query('select * from "Milk_Type"', con=engine)
type2_df = type2_df.drop(columns = ['Full_Name' , 'count'])
complete_milk = complete_milk.merge(type2_df, on = 'type')
complete_milk = complete_milk.drop(columns = {'type' })
complete_milk = complete_milk.rename(columns = {'id' : 'Type_ID'})
#complete_milk


Same process as above is followed to insert Milk data from dataframe into Milk table of database

In [81]:
temp_milk = pd.read_sql_query('select * from "Milk"', con=engine)
temp_milk = temp_milk.drop(columns = 'id')
#temp_milk

In [82]:
db_milk = pd.concat([temp_milk, complete_milk]).drop_duplicates(keep=False)
db_milk = db_milk.reset_index(drop=True)   
#db_milk.head()

In [83]:
db_milk.to_sql(name='Milk', con=engine, if_exists='append', index=False)

In [84]:
pd.read_sql_query('select * from "Milk"', con=engine)

,id,product_id,Brand,features,size,category,image,Store_ID,Type_ID,name
0,1,1.111040e+09,Kroger,none,1 gal,['Dairy'],https://www.kroger.com/product/images/xlarge/f...,1,10,Kroger® Vitamin D Whole Milk
1,2,1.111041e+09,Kroger,none,1/2 gal,['Dairy'],https://www.kroger.com/product/images/xlarge/f...,1,10,Kroger® Vitamin D Whole Milk
2,3,1.111043e+09,Simple Truth Organic,none,0.5 gal,"['Dairy', 'Natural & Organic']",https://www.kroger.com/product/images/xlarge/f...,1,10,Simple Truth Organic® Whole Milk
3,4,1.111043e+09,Simple Truth Organic,none,1 gal,"['Dairy', 'Natural & Organic']",https://www.kroger.com/product/images/xlarge/f...,1,10,Simple Truth Organic™ Whole Milk
4,5,8.132670e+10,A2 Milk,none,59 fl oz,['Dairy'],https://www.kroger.com/product/images/xlarge/f...,1,10,A2 Ultra-Pasteurized Whole Milk
...,...,...,...,...,...,...,...,...,...,...
598,599,3.981542e+06,Iconic,None,None,Dairy & Eggs/Milk,https://images.heb.com/is/image/HEBGrocery/prd...,2,9,Kids Vanilla Vacay Protein Drink
599,600,3.981541e+06,Iconic,None,None,Dairy & Eggs/Milk,https://images.heb.com/is/image/HEBGrocery/prd...,2,9,Kids Fruity Fiesta Protein Drink
600,601,1.290139e+06,Nestle,None,56.3 oz,Dairy & Eggs/Milk,https://images.heb.com/is/image/HEBGrocery/prd...,2,8,NIDO Kinder 1+ Powdered Milk Beverage
601,602,1.148619e+06,Nestle,[' LOW SODIUM'],12.6 oz,Dairy & Eggs/Milk,https://images.heb.com/is/image/HEBGrocery/prd...,2,8,NIDO Kinder 1+ Powdered Milk Beverage


Milk data is pulled from database Milk table and merged with price history. Uneeded price history columns are dropped and renamed to coordinate with Price History Table. This sets the product ID in price history.

In [85]:
price_history_df = kroger_data_df[['Date', 'price.regular', 'price.promo', 'productId']].copy()
heb_price = heb_data_df[['price' , 'coupon', 'id', 'Date']]
heb_price =heb_price.rename(columns={'Date':'date', 'coupon':'saleprice', 'id':'product_id'})
price_history_df =price_history_df.rename(columns={'Date':'date', 'price.regular':'price', 'price.promo':'saleprice', 'productId': 'product_id'})
price_history_df = pd.concat([price_history_df, heb_price])
price_history_df['date'] = pd.to_datetime(price_history_df['date'])
price_history_df

,date,price,saleprice,product_id
0,2021-02-08,2.99,0.0,1111040101
1,2021-02-08,1.99,0.0,1111041600
2,2021-02-08,2.99,0.0,1111041660
3,2021-02-08,1.99,0.0,1111040601
4,2021-02-08,2.99,0.0,1111042315
...,...,...,...,...
255,2021-02-10,3.07,0.0,314131
256,2021-02-10,3.07,0.0,314124
257,2021-02-10,3.07,0.0,314128
258,2021-02-10,3.28,0.0,658779


In [86]:
milk2_df = pd.read_sql_query('select * from "Milk"', con=engine)
milk2_df = milk2_df.drop(columns = ['Brand' , 'size', 'image', 'Store_ID', 'Type_ID', 'category', 'features', 'name'])
merged_price = price_history_df.merge(milk2_df, on = 'product_id')
#merged_price

In [87]:
merged_price = merged_price.drop(columns = ['product_id'])
merged_price = merged_price.rename(columns = {'id' : 'product_id'})
#merged_price

Price History Dataframe is then added to Price History Table. Duplicates are not checked for in this instance because duplicates will trigger an error and stop the update process. 

In [88]:
temp_history = pd.read_sql_query('select * from "Price_History"', con=engine)
db_history = pd.concat([temp_history, merged_price]).drop_duplicates(keep=False)
db_history = db_history.reset_index(drop=True)   
db_history

,product_id,date,saleprice,price


In [89]:
db_history.to_sql(name='Price_History', con=engine, if_exists='append', index=False)

In [90]:
pd.read_sql_query('select * from "Price_History"', con=engine)

,product_id,date,saleprice,price
0,1,2021-02-08,0.0,2.99
1,78,2021-02-08,0.0,1.99
2,79,2021-02-08,0.0,2.99
3,2,2021-02-08,0.0,1.99
4,80,2021-02-08,0.0,2.99
...,...,...,...,...
598,190,2021-02-10,0.0,3.07
599,77,2021-02-10,0.0,3.07
600,191,2021-02-10,0.0,3.07
601,296,2021-02-10,0.0,3.28
